In [1]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [2]:
import torch
import numpy as np

In [3]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from transformers import BertTokenizerFast
Tokenized = BertTokenizerFast.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def Bert_question_answer(question,answer,max_length = 500):

  input_ids = Tokenized.encode(question,answer,max_length = max_length,truncation = True)

  #sep_index = input_ids.index(Tokenized.sep_token_id)

  sep_index = input_ids.index(102)

  len_question = sep_index + 1

  len_passage = len(input_ids)-len_question

  segment_ids = [0]*len_question + [1]*len_passage

  tokens = Tokenized.convert_ids_to_tokens(input_ids)

  start_token_scores = model(torch.tensor([input_ids]),token_type_ids = torch.tensor([segment_ids]))[0]
  end_token_scores = model(torch.tensor([input_ids]),token_type_ids = torch.tensor([segment_ids]))[1]

  answer_start_index = torch.argmax(start_token_scores)
  answer_end_index = torch.argmax(end_token_scores)

  start_token_scores = start_token_scores.detach().numpy().flatten()
  end_token_scores = end_token_scores.detach().numpy().flatten()

  start_token_scores = np.round(start_token_scores[answer_start_index],2)
  end_token_scores = np.round(end_token_scores[answer_end_index],2)

  answer = tokens[answer_start_index]

  for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]


  return (answer_start_index,answer_end_index,start_token_scores,end_token_scores,answer)




In [10]:
passage = 'In a futuristic city named Syntharia, where technology and innovation converge seamlessly, the renowned scientist Dr. Evelyn Harper has just unveiled her latest invention – Quantum Synthesis Engine (QSE). This groundbreaking technology has the potential to reshape the fabric of reality itself, offering unprecedented possibilities for the future of humanity. As the excitement grows, whispers of intrigue and skepticism fill the air, with questions swirling around the mysterious capabilities of QSE.'

question = 'What is the name of the groundbreaking technology unveiled by Dr. Evelyn Harper in the futuristic city of Syntharia?'
print("Question : \n"+ question)

ans = Bert_question_answer(question,passage)
print("Answer is : " , ans, '\n' )

Question : 
What is the name of the groundbreaking technology unveiled by Dr. Evelyn Harper in the futuristic city of Syntharia?
Answer is :  (tensor(54), tensor(56), 8.12, 8.28, 'quantum synthesis engine') 

